In [2]:
# Imports
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
nltk.download('stopwords')


from nltk.tokenize import RegexpTokenizer, word_tokenize
from nltk.corpus import stopwords, wordnet
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.tag import pos_tag
from nltk.probability import FreqDist
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.linear_model import LogisticRegression, LinearRegression, Ridge
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import cross_val_score, cross_val_predict, cross_validate
from tqdm import tqdm

from tensorflow.keras.layers import Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

[nltk_data] Downloading package stopwords to /Users/jf/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
# reading in our cleaned and processed data frame
df_processed = pd.read_csv('../data/processed_a.csv')
# dropping any nulls
df_processed.dropna(inplace=True)

In [10]:
#train test split 
X = df_processed["processed_reviews"]
y = df_processed["price_range"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42
)

#sanity check shape
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((3921,), (1307,), (3921,), (1307,))

In [16]:
y_train

2837    2
1960    3
402     3
3354    2
2337    0
       ..
3100    1
3780    0
5202    1
5237    1
863     2
Name: price_range, Length: 3921, dtype: int64

In [14]:
# The text data is vectorized using the TF-IDF technique, similar to previous models. 
# However, the sparse matrix is converted to a dense matrix, which is a requirement 
# for neural networks in TensorFlow/Keras

vectorizer = TfidfVectorizer(stop_words="english")
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

X_train_vec_dense = X_train_vec.todense()
X_test_vec_dense = X_test_vec.todense()

NameError: name 'x_train_vec_dense' is not defined

In [15]:
X_train_vec_dense

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])

In [18]:
encoder = OneHotEncoder(sparse=False)
y_train_encoded = encoder.fit_transform(y_train.values.reshape(-1, 1))

/Users/jf/anaconda3/envs/learn-env/lib/python3.8/site-packages/sklearn/preprocessing/_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [20]:
# Model architecture
nn = Sequential()
nn.add(Dense(10, activation='relu'))
nn.add(Dropout(0.5))  # Adding dropout with rate 0.5
nn.add(Dense(4, activation='softmax'))

# Compilation
nn.compile(loss='categorical_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])

# Training
nn.fit(X_train_vec_dense, y_train_encoded, epochs=10, batch_size=1, validation_split=0.2);


Epoch 1/10
3136/3136 [==============================] - 11s 4ms/step - loss: 1.3535 - accuracy: 0.3004 - val_loss: 1.2827 - val_accuracy: 0.4433
Epoch 2/10
3136/3136 [==============================] - 10s 3ms/step - loss: 1.2372 - accuracy: 0.3935 - val_loss: 1.2003 - val_accuracy: 0.4739
Epoch 3/10
3136/3136 [==============================] - 10s 3ms/step - loss: 1.1520 - accuracy: 0.4321 - val_loss: 1.1563 - val_accuracy: 0.4662
Epoch 4/10
3136/3136 [==============================] - 10s 3ms/step - loss: 1.0838 - accuracy: 0.4742 - val_loss: 1.1318 - val_accuracy: 0.4904
Epoch 5/10
3136/3136 [==============================] - 10s 3ms/step - loss: 1.0166 - accuracy: 0.5102 - val_loss: 1.1114 - val_accuracy: 0.5070
Epoch 6/10
3136/3136 [==============================] - 10s 3ms/step - loss: 0.9661 - accuracy: 0.5475 - val_loss: 1.1091 - val_accuracy: 0.4968
Epoch 7/10
3136/3136 [==============================] - 10s 3ms/step - loss: 0.9037 - accuracy: 0.5928 - val_loss: 1.1140 - val_ac